# Astronomical Source Detection of the f105w band image of the Hubble Ultra Deep Field

## Prepping the image

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sep
from astropy.io import fits
from matplotlib import rcParams
from matplotlib.patches import Ellipse

rcParams['figure.figsize'] = [10., 8.]

### Open FITS file and store in a 2D array

In [ ]:
fname = "hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits"
hdu_list = fits.open(fname)
image_data = fits.getdata(fname)

### To check the dimensions of the image

In [ ]:
print(type(image_data))
print(image_data.shape)

### Show the image

In [ ]:
image_data_mean = np.mean(image_data)
image_data_std = np.std(image_data)
plt.imshow(image_data, 
           cmap = 'magma', 
           interpolation = 'nearest', 
           vmin = image_data_mean - image_data_std, 
           vmax = image_data_mean + image_data_std, 
           origin = 'lower')
plt.colorbar()
plt.savefig('f105w.png')

## Background subtraction

### Return a background object that holds information on spatially varying background and noise level

In [ ]:
#measure a spatially varying background on the image
image_data = image_data.byteswap().newbyteorder()
bkg = sep.Background(image_data)

### Get a "global" mean and noise of the image background

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

### Evaluate background as 2D array (image is same size as original

In [ ]:
bkg_image = np.array(bkg)

### Show background

In [ ]:
plt.imshow(bkg_image,
           interpolation = 'nearest',
           cmap = 'magma',
           origin = 'lower')
plt.colorbar()
#plt.savefig('f105w_bkg.png')

### Evaluate background noise as 2D array

In [ ]:
bkg_rms = bkg.rms()

### Show background noise

In [ ]:
plt.imshow(bkg_rms, 
           interpolation = 'nearest', 
           cmap = 'magma',
           origin = 'lower')
plt.colorbar()
#plt.savefile('f105w_noise.png')

### Subtract background

In [ ]:
data_sub = image_data - bkg

## Object detection

### Using background-subtracted data to run object detection...

### Background noise is flat, so set detection threshold as 1.5σ where σ is the global background RMS

In [ ]:
σ = bkg.globalrms
objects = sep.extract(data_sub, 1.5, err = σ)

### Check to see how many objects were detected

In [ ]:
object_count = len(objects)
object_count

### To check where the detected objects are, over-plot basic coordinates with some shape parameters

### Plot background-subtracted image with subplots

In [ ]:
fig, ax = plt.subplots()
data_sub_mean = np.mean(data_sub)
data_sub_std = np.std(data_sub)
bkg_sub_image = ax.imshow(data_sub,
                          interpolation = 'nearest',
                          cmap = 'magma',
                          vmin = data_sub_mean - data_sub_std,
                          vmax = data_sub_mean + data_sub_std,
                          origin = 'lower')

#plot an ellipse for each object
for i in range(object_count):
    
    # objects['x'] and objects['y'] gives centroid coordinates of the objects
    # ellipse parameters for each object
    
    x = objects['x'][i]
    y = objects['y'][i]
    width = 6*objects['a'][i]
    height = 6*objects['b'][i]
    angle = objects['theta'][i] * 180./np.pi
    e = Ellipse(xy = (x, y),
                width = width,
                height = height,
                angle = angle)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
    
#plt.savefig('f105w_final.png')

### [objects] has other fields which give information liek second moments and peak pixel positions and values. To see available fields:

In [ ]:
objects.dtype.names

## Aperture photometry

### To perfrom simple circular photometry with a 3 pixel radius at object locations

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub,
                                     objects['x'],
                                     objects['y'],
                                     3.0,
                                     err = σ,
                                     gain = 1.0)

#flux, fluxerr, and flag are 1D arrays with one entry per object

### To show first 10 objects results

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

### Using ABMag to make the histogram (referenced from STSCI website)

In [ ]:
#log cannot have negative in the argument so
flux_1 = flux[flux > 0]
ABMag = -2.5*np.log10(flux_1) - 26.0974

In [ ]:
histogram = plt.hist(ABMag, bins = 'auto')